In [ ]:
### INIT Pyside and ScriptEdit ###

from PySide6.QtWidgets import *
%gui qt
app = QApplication.instance()  # Reuse the existing instance if available
if app is None:  # If no instance exists, create one
    app = QApplication([])

# Create a simple Qt widget
from pylive.QtScriptEditor.script_edit import ScriptEdit
script_edit = ScriptEdit()
script_edit.show()

setTabSize 4


In [ ]:
# set the script
from textwrap import dedent
script_edit.setPlainText(dedent("""\
def hello():
    print("boom")
"""))

In [ ]:
# from pylive.QtScriptEditor.components.invisibles_highlighter import InvisiblesHighlighter

from pylive.QtScriptEditor.components.simple_python_highlighter import SimplePythonHighlighter
script_edit.highlighter = SimplePythonHighlighter(script_edit.document())

In [7]:
from pylive.QtScriptEditor.components.pygments_syntax_highlighter import PygmentsSyntaxHighlighter
script_edit.highlighter = PygmentsSyntaxHighlighter(script_edit.document())